In [ ]:
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [ ]:
root_data = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants')
train = pl.read_csv(root_data / 'train.csv')
test = pl.read_csv(root_data / 'test.csv')

In [ ]:
import polars as pl
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pl.DataFrame):
            X = pl.DataFrame(X)
            
        columns_to_drop = [col for col in self.columns_to_drop if col in X.columns]
        
        return X.drop(columns_to_drop)

class StringParser(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        parts = [s.split('-') for s in X]
        return pl.DataFrame({
            'selection': [p[1] for p in parts],
            'exploration': [float(p[2]) for p in parts],
            'playout': [p[3] for p in parts],
            'score_bounds': [1 if p[4] == 'true' else 0 for p in parts]
        })

class AgentEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.parser = StringParser()
        self.selection_ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
        self.playout_ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
        self.exploration_imputer = SimpleImputer(strategy='constant', fill_value=0)
        self.score_bounds_imputer = SimpleImputer(strategy='constant', fill_value=False)

    def fit(self, X, y=None):
        parsed = self.parser.fit_transform(X)
        self.selection_ohe.fit(parsed.select('selection').to_numpy())
        self.playout_ohe.fit(parsed.select('playout').to_numpy())
        self.exploration_imputer.fit(parsed.select('exploration').to_numpy())
        self.score_bounds_imputer.fit(parsed.select('score_bounds').to_numpy())
        return self

    def transform(self, X):
        parsed = self.parser.transform(X)
        selection_encoded = self.selection_ohe.transform(parsed.select('selection').to_numpy())
        playout_encoded = self.playout_ohe.transform(parsed.select('playout').to_numpy())
        exploration_imputed = self.exploration_imputer.transform(parsed.select('exploration').to_numpy())
        score_bounds_imputed = self.score_bounds_imputer.transform(parsed.select('score_bounds').to_numpy())
        
        return np.hstack([selection_encoded, playout_encoded, exploration_imputed, score_bounds_imputed])

    def get_feature_names_out(self):
        return (
            [f'selection_{name}' for name in self.selection_ohe.get_feature_names_out(['selection'])] +
            [f'playout_{name}' for name in self.playout_ohe.get_feature_names_out(['playout'])] +
            ['exploration', 'score_bounds']
        )

class DualAgentEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.agent_encoder = AgentEncoder()

    def fit(self, X, y=None):
        all_agents = X.get_column('agent1').to_list() + X.get_column('agent2').to_list()
        self.agent_encoder.fit(all_agents)
        return self

    def transform(self, X):

        agent1_encoded = self.agent_encoder.transform(X.get_column('agent1').to_list())
        agent2_encoded = self.agent_encoder.transform(X.get_column('agent2').to_list())
        encoded_agents = pl.DataFrame(
            np.hstack([agent1_encoded, agent2_encoded]),
            schema=self.get_feature_names_out()
        )
        retained_columns = [col for col in X.columns if col not in ['agent1', 'agent2','num_draws_agent1','num_losses_agent1','num_wins_agent1']]
        result = pl.concat([X.select(retained_columns), encoded_agents], how='horizontal')
        
        return result

    def get_feature_names_out(self):
        base_names = self.agent_encoder.get_feature_names_out()
        return [f'agent1_{name}' for name in base_names] + [f'agent2_{name}' for name in base_names]


def create_preprocessing_pipeline():
    drop_columns = ColumnDropper(columns_to_drop=['Id', 'Properties', 'Format', 'Time', 'Discrete', 'Realtime', 'Turns', 'Alternating',
        'Simultaneous', 'HiddenInformation', 'Match', 'AsymmetricRules', 'AsymmetricPlayRules',
        'AsymmetricEndRules', 'AsymmetricSetup', 'Players', 'NumPlayers', 'Simulation', 'Solitaire',
        'TwoPlayer', 'Multiplayer', 'Coalition', 'Puzzle', 'DeductionPuzzle', 'PlanningPuzzle', 'Equipment', 
        'Container', 'Board', 'PrismShape', 'ParallelogramShape', 'RectanglePyramidalShape', 'TargetShape', 
        'BrickTiling', 'CelticTiling', 'QuadHexTiling', 'Hints', 'PlayableSites', 'Component', 'DiceD3',
        'BiasedDice', 'Card', 'Domino', 'Rules', 'SituationalTurnKo', 'SituationalSuperko', 'InitialAmount',
        'InitialPot', 'Play', 'BetDecision', 'BetDecisionFrequency', 'VoteDecisionFrequency',
        'ChooseTrumpSuitDecision', 'ChooseTrumpSuitDecisionFrequency', 'LeapDecisionToFriend', 
        'LeapDecisionToFriendFrequency', 'HopDecisionEnemyToFriend', 'HopDecisionEnemyToFriendFrequency',
        'HopDecisionFriendToFriend', 'FromToDecisionWithinBoard', 'FromToDecisionBetweenContainers', 'BetEffect',
        'BetEffectFrequency', 'VoteEffectFrequency', 'SwapPlayersEffectFrequency', 'TakeControl',
        'TakeControlFrequency', 'PassEffectFrequency', 'SetCost', 'SetCostFrequency', 'SetPhase', 
        'SetPhaseFrequency', 'SetTrumpSuit', 'SetTrumpSuitFrequency', 'StepEffectFrequency', 
        'SlideEffectFrequency', 'LeapEffectFrequency', 'HopEffectFrequency', 'FromToEffectFrequency',
        'SwapPiecesEffect', 'SwapPiecesEffectFrequency', 'ShootEffect', 'ShootEffectFrequency', 'MaxCapture',
        'OffDiagonalDirection', 'Information', 'HidePieceType', 'HidePieceOwner', 'HidePieceCount',
        'HidePieceRotation', 'HidePieceValue', 'HidePieceState', 'InvisiblePiece', 'End', 'LineDrawFrequency',
        'ConnectionDraw', 'ConnectionDrawFrequency', 'GroupLossFrequency', 'GroupDrawFrequency', 
        'LoopLossFrequency', 'LoopDraw', 'LoopDrawFrequency', 'PatternLoss', 'PatternLossFrequency', 
        'PatternDraw', 'PatternDrawFrequency', 'PathExtentEndFrequency', 'PathExtentWinFrequency',
        'PathExtentLossFrequency', 'PathExtentDraw', 'PathExtentDrawFrequency', 'TerritoryLoss',
        'TerritoryLossFrequency', 'TerritoryDraw', 'TerritoryDrawFrequency', 'CheckmateLoss', 
        'CheckmateLossFrequency', 'CheckmateDraw', 'CheckmateDrawFrequency', 'NoTargetPieceLoss', 
        'NoTargetPieceLossFrequency', 'NoTargetPieceDraw', 'NoTargetPieceDrawFrequency', 'NoOwnPiecesDraw',
        'NoOwnPiecesDrawFrequency', 'FillLoss', 'FillLossFrequency', 'FillDraw', 'FillDrawFrequency', 
        'ScoringDrawFrequency', 'NoProgressWin', 'NoProgressWinFrequency', 'NoProgressLoss', 
        'NoProgressLossFrequency', 'SolvedEnd', 'Behaviour', 'StateRepetition', 'PositionalRepetition',
        'SituationalRepetition', 'Duration', 'Complexity', 'BoardCoverage', 'GameOutcome', 'StateEvaluation',
        'Clarity', 'Narrowness', 'Variance', 'Decisiveness', 'DecisivenessMoves', 'DecisivenessThreshold', 
        'LeadChange', 'Stability', 'Drama', 'DramaAverage', 'DramaMedian', 'DramaMaximum', 'DramaMinimum', 
        'DramaVariance', 'DramaChangeAverage', 'DramaChangeSign', 'DramaChangeLineBestFit', 'DramaChangeNumTimes', 
        'DramaMaxIncrease', 'DramaMaxDecrease', 'MoveEvaluation', 'MoveEvaluationAverage', 'MoveEvaluationMedian',
        'MoveEvaluationMaximum', 'MoveEvaluationMinimum', 'MoveEvaluationVariance', 'MoveEvaluationChangeAverage',
        'MoveEvaluationChangeSign', 'MoveEvaluationChangeLineBestFit', 'MoveEvaluationChangeNumTimes',
        'MoveEvaluationMaxIncrease', 'MoveEvaluationMaxDecrease', 'StateEvaluationDifference',
        'StateEvaluationDifferenceAverage', 'StateEvaluationDifferenceMedian', 'StateEvaluationDifferenceMaximum', 
        'StateEvaluationDifferenceMinimum', 'StateEvaluationDifferenceVariance', 'StateEvaluationDifferenceChangeAverage',
        'StateEvaluationDifferenceChangeSign', 'StateEvaluationDifferenceChangeLineBestFit', 
        'StateEvaluationDifferenceChangeNumTimes', 'StateEvaluationDifferenceMaxIncrease', 
        'StateEvaluationDifferenceMaxDecrease', 'BoardSitesOccupied', 'BoardSitesOccupiedMinimum', 
        'BranchingFactor', 'BranchingFactorMinimum', 'DecisionFactor', 'DecisionFactorMinimum', 'MoveDistance',
        'MoveDistanceMinimum', 'PieceNumber', 'PieceNumberMinimum', 'ScoreDifference', 'ScoreDifferenceMinimum', 
        'ScoreDifferenceChangeNumTimes', 'Roots', 'Cosine', 'Sine', 'Tangent', 'Exponential', 'Logarithm', 
        'ExclusiveDisjunction', 'Float', 'HandComponent', 'SetHidden', 'SetInvisible', 'SetHiddenCount', 'SetHiddenRotation',
        'SetHiddenState', 'SetHiddenValue', 'SetHiddenWhat', 'SetHiddenWho','GameRulesetName', 'EnglishRules', 'LudRules'])
    

    encode_agents = DualAgentEncoder()

    pipeline = Pipeline(steps=[
        ('drop_columns', drop_columns),
        ('encode_agents', encode_agents),
        ('scaler', StandardScaler())
    ])

    return pipeline




In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

def create_models():
    cat_model = CatBoostRegressor(
        iterations=3500, learning_rate=0.04, depth=9, l2_leaf_reg=0.0005,
        min_data_in_leaf=25, random_strength=0.01, bagging_temperature=0.04,
        grow_policy='SymmetricTree', verbose=0, task_type='GPU'
    )

    lgbm_model = LGBMRegressor(
        learning_rate=0.02, num_leaves=180, n_estimators=3500, min_child_samples=50,
        subsample=0.85, colsample_bytree=0.9, reg_alpha=0.02, reg_lambda=0.001,
        scale_pos_weight=1.2, min_split_gain=1e-4, min_child_weight=0.005,
        verbose=-1, device='gpu'
    )

    voting_regressor = VotingRegressor(
        estimators=[('CAT', cat_model), ('Light', lgbm_model)],
        weights=[0.588196, 0.411804]
    )
    model_pipeline = Pipeline([('model', voting_regressor)])

    return model_pipeline


In [ ]:
full_pipeline = Pipeline([
    ('preprocess', create_preprocessing_pipeline()),
    ('model', create_models())
        ])
full_pipeline

In [ ]:
X = train.drop(['utility_agent1'])
y = train['utility_agent1']
groups = train['GameRulesetName']
group_kfold = GroupKFold(n_splits=5)

cv_scores = cross_val_score(full_pipeline, X, y, groups=groups, cv=group_kfold, scoring='neg_root_mean_squared_error')

print(f"Cross-validation RMSE scores: {cv_scores}")
print(f"Mean CV RMSE: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores) * 2:.4f})")



In [ ]:
import os
import sys
import kaggle_evaluation.mcts_inference_server


preprocess_pipeline = None
model_pipeline = None
full_pipeline = None
counter = 0

target = 'utility_agent1'


def train_model():
    global preprocess_pipeline, model_pipeline, full_pipeline

    comp_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants')
    train = pl.read_csv(comp_path / 'train.csv')

    X = train.drop([target])
    y = train[target]
    groups = train['GameRulesetName']

    preprocess_pipeline = create_preprocessing_pipeline()  
    model_pipeline = create_models()

    full_pipeline = Pipeline([
        ('preprocess', preprocess_pipeline),
        ('model', model_pipeline)
        ])
    full_pipeline.fit(X, y)

def predict(test, submission):
    global counter, full_pipeline

    if counter == 0:
        train_model()
    
    counter += 1

    test_df = test.to_pandas() 
    preds = full_pipeline.predict(test_df)

    submission = submission.with_columns(pl.Series(target, preds))
    print(submission)
    return submission

inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

## About This Notebook 📘

If you found this notebook useful, your upvote is appreciated. You can also connect with me on LinkedIn https://www.linkedin.com/in/thesidsat/

Happy Kaggling :D 